## 📘 How to Use Kaggle (Upload Dataset & Notebook)

### ✅ Step 1: Create Kaggle Account
- Go to 👉 https://www.kaggle.com  
- Sign in using Google / Email

---

### ✅ Step 2: Upload Your Dataset
1. Click **Datasets** → **Create New Dataset**
2. Upload your **dataset folder or ZIP file**
3. Add:
   - Dataset name
   - Short description
4. Set visibility → **Public / Private**
5. Click **Create**

✅ After upload, Kaggle gives a dataset path like:


### 🩺 Anemia Detection from Eye & Nail Images using EfficientNetV2-S + MixStyle + Focal Loss

## 🔍 Project Overview

This notebook trains a **Keras 3–compatible deep learning model** to detect **anemia** from medical images of:

- 👁 Conjunctiva (eye)
- 💅 Fingernails

The pipeline includes:

- **EfficientNetV2-S** backbone for feature extraction  
- **Custom MixStyle layer** for domain generalization  
- **BalancedSequence sampler** to handle class imbalance  
- **Focal loss** to focus on hard examples  
- Stratified Train/Validation/Test split  
- Final evaluation with classification report & confusion matrix  

---

### 📚 Key Libraries (Official Docs)

- TensorFlow / Keras → https://www.tensorflow.org  
- OpenCV → https://opencv.org  
- NumPy → https://numpy.org  
- Pandas → https://pandas.pydata.org  
- Matplotlib → https://matplotlib.org  
- Seaborn → https://seaborn.pydata.org  
- scikit-learn → https://scikit-learn.org  

### 🟢 Section 1: Imports, Reproducibility & TensorFlow Version

In this section we:

- Import core Python & ML libraries:
  - `os`, `random`, `math`, `gc` → utilities & memory handling
  - `numpy`, `pandas` → numerical computation & tabular data  
    - NumPy docs: https://numpy.org  
    - Pandas docs: https://pandas.pydata.org  
  - `cv2` → image loading & processing  
    - OpenCV docs: https://docs.opencv.org  
  - `matplotlib`, `seaborn` → plotting & visualization  
    - Matplotlib: https://matplotlib.org  
    - Seaborn: https://seaborn.pydata.org  
  - `tensorflow.keras` → model definition & training  
    - TensorFlow/Keras: https://www.tensorflow.org  
  - `sklearn` → splitting and evaluation  
    - scikit-learn: https://scikit-learn.org  

- Set a fixed random seed (`SEED = 42`) for:
  - `random`
  - `numpy`
  - `tf.random`



In [ ]:
# ==============================================================
#   FINAL KERAS-3 COMPATIBLE ANEMIA DETECTION TRAINING NOTEBOOK
#   EfficientNetV2-S + MixStyle + BalancedSampler + Focal Loss
# ==============================================================

import os, random, math, gc
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

print("TF VERSION:", tf.__version__)



### 🟢 Section 2: Dataset Paths & File Collection

Here we define the directory structure for our anemia dataset:

- **Conjunctiva (eye) images**  
  - Anemic: `"Anemia Detection/Anemic"`  
  - Non-Anemic: `"Anemia Detection/Non Anemia"`  

- **Nail images**  
  - Anemic: `"Fingernails/Anemic"`  
  - Non-Anemic: `"Fingernails/NonAnemic"`  

Using the helper function `collect_paths()` we:

- Scan each folder for image files (`.png`, `.jpg`, `.jpeg`)  
- Build a list of dictionaries with:
  - `image_path`
  - `label` (1 = anemic, 0 = non-anemic)
  - `source` (`"conj"` or `"nail"`)

Then we construct a **Pandas DataFrame** `df` from this list and print:

- Total number of images
- Number of images per source type (eye vs nail)

This gives us a **single unified metadata table** for both modalities.


**Dataset Path**: https://www.kaggle.com/datasets/huebitsvizg/anemia-detection

In [ ]:
# 1. Dataset Paths (Modify if needed)
paths = {
    "conj_anemic":     "/kaggle/input/anemia-using-eyes/Anemia Detection/Anemic",
    "conj_nonanemic":  "/kaggle/input/anemia-using-eyes/Anemia Detection/Non Anemia",
    "nail_anemic":     "/kaggle/input/anemia-detection-nails/Fingernails/Anemic",
    "nail_nonanemic":  "/kaggle/input/anemia-detection-nails/Fingernails/NonAnemic",
}

In [ ]:
# 2. Load Filepaths Into a DataFrame
def collect_paths(folder, label, source):
    rows=[]
    if not os.path.exists(folder):
        print("Missing:", folder)
        return rows
    for f in os.listdir(folder):
        if f.lower().endswith(('.png','.jpg','.jpeg')):
            rows.append({"image_path": os.path.join(folder,f),
                         "label": label,
                         "source": source})
    return rows

data = []
data += collect_paths(paths["conj_anemic"], 1, "conj")
data += collect_paths(paths["conj_nonanemic"], 0, "conj")
data += collect_paths(paths["nail_anemic"], 1, "nail")
data += collect_paths(paths["nail_nonanemic"], 0, "nail")

df = pd.DataFrame(data)
print("Total images:", len(df))
print(df.source.value_counts())


## 🟢 Section 3: Select Image Modality (Conjunctiva or Nails)

Our pipeline supports **two different input types**:

- `"conj"` → conjunctiva (eye) images  
- `"nail"` → fingernail images  

We select a modality using:

```python
modality = "conj"   # or "nail"
df_mod = df[df.source == modality].reset_index(drop=True)


In [ ]:
# 3. Choose ONE Modality (conj OR nail)
modality = "conj"   # or "nail"
df_mod = df[df.source==modality].reset_index(drop=True)
print("Using modality:", modality, "| Images:", len(df_mod))




---

### 🟦 Markdown Cell  — Image Preprocessing (WB + CLAHE)

```markdown
## 🟢 Section 4: Image Preprocessing – White Balance & CLAHE

To reduce the impact of lighting, skin tone, and acquisition device differences, we define a robust preprocessing pipeline:

1. **Gray-World White Balance (`white_balance_grayworld`)**
   - Estimates average color of the image
   - Adjusts each channel so overall color balance is neutral
   - Helps correct global color cast (e.g., yellowish/blueish tint)

2. **CLAHE in LAB Color Space (`apply_clahe_rgb`)**
   - Converts BGR → LAB
   - Applies **Contrast Limited Adaptive Histogram Equalization** (CLAHE) on the L (lightness) channel  
     - OpenCV CLAHE docs: https://docs.opencv.org/ → search *CLAHE*
   - Enhances local contrast while controlling noise

3. **Final Preprocessing (`preprocess_image`)**
   - Read image from path with `cv2.imread`
   - Apply white balance + CLAHE
   - Resize to `(380, 380)`
   - Convert BGR → RGB
   - Normalize to `[0, 1]` and cast to `float32`

This preprocessing pipeline is crucial in **medical imaging**, where subtle color variations may indicate anemia.


In [ ]:
# 4. Preprocessing Functions (White balance + CLAHE)
def white_balance_grayworld(img):
    img = img.astype(np.float32)
    avg = np.mean(img)
    for c in range(3):
        img[:,:,c] *= avg/(np.mean(img[:,:,c])+1e-6)
    return np.clip(img,0,255).astype(np.uint8)

def apply_clahe_rgb(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0)
    l = clahe.apply(l)
    lab = cv2.merge((l,a,b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def preprocess_image(path, size=(380,380)):
    img = cv2.imread(path)
    if img is None:
        return None
    img = white_balance_grayworld(img)
    img = apply_clahe_rgb(img)
    img = cv2.resize(img, size)
    img = img[:,:,::-1]/255.0  # RGB
    return img.astype(np.float32)

### 🟢 Section 5: MixStyle Layer for Domain Generalization

This section implements a custom **MixStyle** Keras layer, inspired by domain generalization techniques:

- Works **only during training** (checked via `training` flag)
- Randomly perturbs **feature statistics** (mean and std) across samples in a batch
- Encourages the model to focus on **shape & structure**, not just specific color distributions

Key Steps:

- Compute per-sample mean and variance over spatial dimensions
- Normalize features
- Randomly permute batch indices and mix statistics (`mu`, `sigma`)
- Reconstruct features with mixed style statistics

Keras custom layers reference:  
https://www.tensorflow.org/guide/keras/custom_layers_and_models  

MixStyle helps the model generalize better to unseen lighting conditions, cameras, or skin tones.


In [ ]:
# 5. MixStyle (Training-only)
class MixStyle(layers.Layer):
    def __init__(self, p=0.5, **kwargs):
        super().__init__(**kwargs)
        self.p = p

    def call(self, x, training=False):
        if not training:
            return x

        # sample once per batch
        apply_mix = tf.random.uniform([])

        # Use TF conditional instead of Python if/else
        return tf.cond(
            apply_mix < self.p,
            lambda: self._mix(x),
            lambda: x
        )

    def _mix(self, x):
        mu = tf.reduce_mean(x, axis=[1,2], keepdims=True)
        var = tf.math.reduce_variance(x, axis=[1,2], keepdims=True)
        sigma = tf.sqrt(var + 1e-6)

        x_norm = (x - mu) / sigma

        bs = tf.shape(x)[0]
        perm = tf.random.shuffle(tf.range(bs))

        mu2    = tf.gather(mu, perm)
        sigma2 = tf.gather(sigma, perm)

        lam = tf.random.uniform((bs,1,1,1))

        mu_mix  = lam * mu + (1-lam) * mu2
        sigma_mix = lam * sigma + (1-lam) * sigma2

        return x_norm * sigma_mix + mu_mix

    def get_config(self):
        return {"p": self.p}


### 🟢 Section 6: BalancedSequence – Class-Balanced Data Generator

This section defines `BalancedSequence`, a custom **Keras `Sequence`** that:

- Ensures **balanced batches**:
  - Half of each batch are label `0` (non-anemic)
  - Half are label `1` (anemic)
- Loads images using `preprocess_image()`
- Applies simple augmentation (random horizontal flip) when `aug=True`
- Applies **EfficientNetV2 preprocessing** (`eff_v2_pre`) **outside** the model graph  
  - EfficientNetV2 Keras docs:  
    https://keras.io/api/applications/efficientnet_v2/

Why use `Sequence`?

- Keras `Sequence` is:
  - Thread-safe
  - Works well with `model.fit`
  - Recommended for large datasets in TF/Keras 3

Balanced sampling is particularly important in **medical applications** where disease-positive cases may be relatively rare.


In [ ]:
# 6. BalancedSequence — SAFE VERSION
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as eff_v2_pre

class BalancedSequence(Sequence):
    def __init__(self, df, batch=32, size=(380,380), aug=True):
        self.df = df.reset_index(drop=True)
        self.batch = batch
        self.size = size
        self.aug = aug

        self.cls_idx = {
            0: self.df.index[self.df.label==0].tolist(),
            1: self.df.index[self.df.label==1].tolist()
        }

    def __len__(self):
        return math.ceil(len(self.df)/self.batch)

    def __getitem__(self, idx):
        imgs=[]
        labels=[]

        # equal samples per class
        per_class = self.batch//2

        for c in [0,1]:
            for _ in range(per_class):
                i = random.choice(self.cls_idx[c])
                row = self.df.loc[i]
                img = preprocess_image(row.image_path, self.size)
                if img is None: continue
                if self.aug and random.random()<0.5:
                    img = np.fliplr(img)
                imgs.append(img)
                labels.append([float(c)])

        imgs    = np.array(imgs)
        labels  = np.array(labels)

        # preprocess_input here (NOT inside model)
        imgs = eff_v2_pre(imgs*255.0)

        return imgs, labels


### 🟢 Section 7: Model Architecture – EfficientNetV2-S + MixStyle Head

We now define the Keras model:

- **Backbone**: `EfficientNetV2S` (pretrained on ImageNet)
  - `include_top=False` → remove default classifier
  - Input shape = `(380, 380, 3)`
  - Official Keras docs:  
    https://keras.io/api/applications/efficientnet_v2/

- **Custom Classification Head**:
  1. `MixStyle` layer applied to backbone feature maps
  2. `GlobalAveragePooling2D` to convert feature maps to a vector
  3. Dense(256, ReLU) + Dropout(0.5)
  4. Dense(1, sigmoid) for **binary classification** (anemic vs non-anemic)

This design leverages strong pretrained features while still allowing the model to learn anemia-specific patterns.


In [ ]:
# 7. Build Keras-3 Safe Model (NO preprocess inside graph)
IMG_SIZE = (380,380,3)

def build_model():
    inp = layers.Input(shape=IMG_SIZE)
    base = tf.keras.applications.EfficientNetV2S(
        include_top=False,
        weights="imagenet",
        input_tensor=inp
    )
    x = MixStyle(p=0.5)(base.output)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    return models.Model(inp, out)

model = build_model()
model.summary()


### 🟢 Section 8: Focal Loss for Imbalanced Binary Classification

Here we define **Focal Loss**, which improves learning in class-imbalanced settings:

- Extends Binary Cross-Entropy by down-weighting **easy** examples
- Focuses training on **hard, misclassified** examples
- Parameters:
  - `gamma` (typically 2.0) → focusing parameter
  - `alpha` (typically 0.25) → weight for positive/negative balance

Original focal loss paper (for reference):  
https://arxiv.org/abs/1708.02002  

This loss is widely used in medical imaging, object detection, and rare event classification tasks.


In [ ]:
# 8. Focal Loss
def focal_loss(y_true,y_pred,gamma=2.0,alpha=0.25):
    y_true = tf.cast(y_true, tf.float32)
    eps=1e-7
    y_pred=tf.clip_by_value(y_pred,eps,1-eps)
    bce=-(y_true*tf.math.log(y_pred)+(1-y_true)*tf.math.log(1-y_pred))
    w=alpha*tf.pow(1-y_pred,gamma)*y_true + (1-alpha)*tf.pow(y_pred,gamma)*(1-y_true)
    return tf.reduce_mean(w*bce)


### 🟢 Section 9: Train / Validation / Test Split

In this section we:

- Split `df_mod` (chosen modality) into:
  - 80% → train+val
  - 20% → test (held out)
- Further split the training part:
  - 90% → actual training set
  - 10% → validation set

We use `train_test_split` from scikit-learn with:

- `stratify=df_mod.label` to preserve class balance
- `random_state=SEED` for reproducibility

scikit-learn train_test_split docs:  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html  

Indices are reset to avoid indexing issues later when building generators.


In [ ]:
# 9. Train / Val / Test Split
train_df, test_df = train_test_split(df_mod, test_size=0.20,
                                     stratify=df_mod.label, random_state=SEED)

train_df, val_df = train_test_split(train_df, test_size=0.10,
                                    stratify=train_df.label, random_state=SEED)

# FIX
train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

print("Train:",len(train_df),"Val:",len(val_df),"Test:",len(test_df))


### 🟢 Section 10: Create Balanced Train & Validation Sequences

We now build the actual data generators:

```python
train_seq = BalancedSequence(train_df, batch=32, aug=True)
val_seq   = BalancedSequence(val_df,   batch=32, aug=False)


In [ ]:
# 10. Create Sequences
train_seq = BalancedSequence(train_df, batch=32, aug=True)
val_seq   = BalancedSequence(val_df,   batch=32, aug=False)



### 🟢 Section 11: Initial Training (Frozen Backbone)

Here we:

- Compile the model with:
  - Optimizer: `Adam(1e-4)`
  - Loss: `focal_loss`
  - Metric: `"accuracy"`
- Define callbacks:
  - `ReduceLROnPlateau` → reduce learning rate when `val_loss` plateaus  
    Docs: https://keras.io/api/callbacks/reduce_lr_on_plateau/  
  - `EarlyStopping` → stop early if `val_loss` stops improving  
    Docs: https://keras.io/api/callbacks/early_stopping/

This stage usually trains primarily the **top layers** and stabilizes the classifier before full fine-tuning.


In [ ]:
# 11. Compile & Train (HEAD ONLY)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=focal_loss,
    metrics=["accuracy"]
)

cb = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=2),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=4,restore_best_weights=True),
]

history = model.fit(train_seq, validation_data=val_seq, epochs=4, callbacks=cb)


### 🟢 Section 12: Fine-Tuning the Last 40 Layers

After initial training, we:

- Unfreeze the **last 40 layers** of the model:
  ```python
  for layer in model.layers[-40:]:
      layer.trainable = True


In [ ]:
# 12. FINE-TUNE LAST 40 LAYERS
for layer in model.layers[-40:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=focal_loss,
    metrics=["accuracy"]
)

history2 = model.fit(train_seq, validation_data=val_seq, epochs=6, callbacks=cb)



### 🟢 Section 13: Test Set Evaluation & Model Export

In the final section we:

1. Loop over all images in `test_df`
2. Apply `preprocess_image()` + EfficientNetV2 preprocessing (`eff_v2_pre`)
3. Use `model.predict()` to obtain probabilities
4. Convert probabilities to binary predictions (`pred > 0.5`)
5. Compute:
   - `classification_report` (precision, recall, F1-score)  
     Docs: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html  
   - `confusion_matrix` to inspect true/false positives/negatives  
     Docs: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html  

6. Save the final model as:
```python
/kaggle/working/effnetv2s_balance_final.keras


In [ ]:
# 13. EVALUATE ON TEST SET
X=[]; Y=[]
for _,row in test_df.iterrows():
    img = preprocess_image(row.image_path,(380,380))
    X.append(img); Y.append(row.label)

X = np.array(X)
X = eff_v2_pre(X*255.0)
Y = np.array(Y)

pred = model.predict(X)
pred_bin = (pred>0.5).astype(int).ravel()

print(classification_report(Y,pred_bin,digits=4))
print(confusion_matrix(Y,pred_bin))


In [ ]:
model.save("/kaggle/working/effnetv2s_balance_final.keras")
print("Model saved!")


# Run this in colab


## 🩺 MedVision: AI-Based Anemia Detection Web App (Flask + EfficientNetV2)

### 📌 Project Overview

This project deploys a **deep learning–based anemia detection system** using:
- A trained **EfficientNetV2-S model**
- Medical image preprocessing (White Balance + CLAHE)
- **Flask** for web deployment
- **ngrok** for public access from Google Colab

Users upload an eye or nail image → the system predicts:
- ✅ Non-Anemic  
- 🚨 Anemic  
along with a **confidence score**.

---

### ✅ Technologies Used (Official Links)

- TensorFlow / Keras → https://www.tensorflow.org  
- Flask → https://flask.palletsprojects.com  
- OpenCV → https://opencv.org  
- NumPy → https://numpy.org  
- Pillow → https://python-pillow.org  
- ngrok → https://ngrok.com  
- Google Colab → https://colab.research.google.com  


####  Cell 1: Install Dependencies & Mount Google Drive

This cell prepares the **deployment environment** by:
- Installing required libraries for deep learning and web deployment
- Mounting Google Drive to load the trained model
- Creating folders for HTML templates and static files

---

####  Install Required Libraries

In [ ]:
!pip install flask pyngrok opencv-python pillow numpy tensorflow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p templates static


---
##  Cell 2: Flask Backend & Model Loading

This cell creates the **Flask backend application** and:
- Loads the trained **EfficientNetV2-S anemia model**
- Registers the custom `MixStyle` layer for compatibility
- Prepares upload directory and image size configuration

---

###  Model Path Configuration

```python
MODEL_PATH = "/content/drive/My Drive/Anemia Detection/effnetv2s_balance_final.keras"


In [ ]:
%%writefile app.py
import os
import numpy as np
import cv2
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as effv2_pre

# ---------------------------
# CONFIG
# ---------------------------
MODEL_PATH = "/content/drive/My Drive/Anemia Detection/effnetv2s_balance_final.keras"
UPLOAD_FOLDER = "static/uploads"
IMG_HW = 380
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# ---------------------------
# Minimal MixStyle (NO-OP for inference)
# ---------------------------
class MixStyle(layers.Layer):
    def __init__(self, p=0.5, **kwargs):
        super().__init__(**kwargs)
        self.p = p

    def call(self, x, training=False):
        return x

    def get_config(self):
        return {"p": self.p}

# ---------------------------
# LOAD MODEL
# ---------------------------
print("Loading model:", MODEL_PATH)

model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={"MixStyle": MixStyle},
    compile=False
)

print("Model Loaded Successfully.")

# ---------------------------
# PREPROCESSING
# ---------------------------
def white_balance_grayworld(img):
    img = img.astype(np.float32)
    avg = (img[:,:,0].mean() + img[:,:,1].mean() + img[:,:,2].mean())/3.0 + 1e-6
    for c in range(3):
        img[:,:,c] *= avg/(img[:,:,c].mean()+1e-6)
    return np.clip(img,0,255).astype(np.uint8)

def apply_clahe_rgb(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0)
    l2 = clahe.apply(l)
    return cv2.cvtColor(cv2.merge((l2,a,b)), cv2.COLOR_LAB2BGR)

def preprocess_for_model(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Error reading image.")

    img = white_balance_grayworld(img)
    img = apply_clahe_rgb(img)
    img = cv2.resize(img, (IMG_HW, IMG_HW))

    img_rgb = img[:, :, ::-1]  # BGR → RGB
    img_norm = img_rgb.astype(np.float32) / 255.0

    x = effv2_pre(np.expand_dims(img_norm * 255.0, axis=0))
    return x, img_rgb

# ---------------------------
# FLASK
# ---------------------------
app = Flask(__name__)
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    if "image" not in request.files:
        return "No file uploaded", 400

    file = request.files["image"]
    if file.filename == "":
        return "Empty filename", 400

    filename = secure_filename(file.filename)
    save_path = os.path.join(UPLOAD_FOLDER, filename)
    file.save(save_path)

    # preprocess
    x, _ = preprocess_for_model(save_path)

    # prediction
    pred = float(model.predict(x)[0][0])
    label = "Anemic" if pred >= 0.5 else "Non-Anemic"
    confidence = round(pred if pred >= 0.5 else 1 - pred, 4)


    return render_template(
        "result.html",
        image_path="/" + save_path,
        result=label,
        confidence=confidence
    )

# ---------------------------
# RUN
# ---------------------------
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)



### 🟢 Cell : Frontend Upload Interface (index.html)

This page provides the **user interface** for:

- Uploading a medical image
- Submitting it for AI analysis

---

#### ✅ UI Features

- Glass-morphism medical card layout
- Image upload box
- “Analyze Image” button
- Responsive mobile-friendly design

The page uses:
- HTML for structure  
- CSS for styling  

✅ This is the **entry point of the AI diagnostic system**.


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MedVision • Anemia Detection</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body class="bg-med">

<div class="center-container">
    <div class="glass-card">

        <h1 class="title">MedVision Diagnostics</h1>
        <p class="subtitle">AI-Powered Non-Invasive Anemia Screening</p>

        <form action="/predict" method="POST" enctype="multipart/form-data">

            <label class="upload-box">
                <span class="upload-text">Tap or Click to Upload an Image</span>
                <input type="file" name="image" accept="image/*" required>
            </label>

            <button type="submit" class="primary-btn">Analyze Image</button>
        </form>

    </div>
</div>

</body>
</html>


### 🟢 Cell : Diagnosis Result Page (result.html)

This page displays:

- Uploaded medical image
- Final diagnosis:
  - ✅ Non-Anemic
  - 🚨 Anemic
- Confidence percentage
- Visual confidence progress bar
- Button to return to the home page

✅ This provides a clean, understandable **medical-style report view** for users.


In [ ]:
%%writefile templates/result.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MedVision • Diagnosis Result</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body class="bg-med">

<div class="center-container">
    <div class="glass-card">

        <h1 class="title">Diagnosis Result</h1>

        <img src="{{ image_path }}" class="result-image">

        <div class="result-section">
            <p class="result-label">Diagnosis</p>
            <p class="result-value">{{ result }}</p>

            <p class="confidence-label">Confidence</p>

            <div class="confidence-bar">
                <div class="confidence-fill" style="width: {{ confidence }}%;"></div>
            </div>

            <p class="confidence-value">{{ confidence }}%</p>
        </div>

        <a href="/" class="secondary-btn back-home">← Back to Home</a>

    </div>
</div>

</body>
</html>


### 🟢 Cell : Web Application Styling (CSS)

This file defines the full **visual theme** of the MedVision web app:

- Gradient medical background
- Glass-effect UI cards
- Upload box hover effects
- Diagnosis result highlights
- Confidence progress bar
- Mobile-responsive layout

✅ The design focuses on:
- Clean medical aesthetics
- Accessibility
- Professional look


In [ ]:
%%writefile static/style.css


/* ---------- GLOBAL ---------- */
body {
    margin: 0;
    padding: 0;
    background: linear-gradient(145deg, #e6efff, #f8fbff);
    font-family: 'Segoe UI', sans-serif;
}

/* ---------- CENTER EVERYTHING ---------- */
.center-container {
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 100vh;
    padding: 20px;
}

/* ---------- CARD ---------- */
.glass-card {
    width: 100%;
    max-width: 420px;
    padding: 30px 28px;
    background: rgba(255, 255, 255, 0.75);
    backdrop-filter: blur(14px);
    border-radius: 20px;
    box-shadow: 0px 8px 30px rgba(0, 0, 0, 0.12);
    text-align: center;
}

/* ---------- TEXT ---------- */
.title {
    font-size: 27px;
    font-weight: 800;
    color: #0a3d7e;
    margin-bottom: 6px;
}

.subtitle {
    color: #4d77aa;
    font-size: 15px;
    margin-bottom: 22px;
}

/* ---------- UPLOAD BOX ---------- */
.upload-box {
    width: 90%;
    max-width: 500px;         /* limits width */
    margin: 0 auto 25px auto; /* centers element */
    padding: 30px 20px;
    border: 2px dashed #7aa6ff;
    background: #f2f6ff;
    border-radius: 18px;
    cursor: pointer;
    transition: 0.25s;
    display: flex;
    justify-content: center;
    align-items: center;
    text-align: center;
}

.upload-box:hover {
    border-color: #3f7bfd;
    background: #e8efff;
}

.upload-text {
    font-weight: 600;
    color: #315a9d;
    font-size: 18px;
}

.upload-box input {
    display: none;
}

/* ---------- BUTTON ---------- */
.primary-btn {
    width: 100%;
    padding: 14px;
    border: none;
    border-radius: 12px;
    font-size: 17px;
    font-weight: 600;
    background: #1e5af7;
    color: white;
    cursor: pointer;
    transition: 0.25s;
    margin-top: 8px;
}

.primary-btn:hover {
    background: #1444c5;
}

/* ---------- RESULT PAGE ---------- */
.result-image {
    width: 55%;
    margin-top: 20px;
    border-radius: 14px;
    box-shadow: 0px 6px 18px rgba(0, 0, 0, 0.15);
}

.result-label {
    text-align: left;
    margin-top: 20px;
    font-size: 17px;
    font-weight: 600;
    color: #0a3d7e;
}

.result-value {
    font-size: 22px;
    color: #1e5af7;
    font-weight: 800;
    margin-bottom: 14px;
}

/* ---------- CONFIDENCE BAR ---------- */
.confidence-bar {
    width: 100%;
    height: 12px;
    background: #d6e3ff;
    border-radius: 20px;
    margin-top: 5px;
    overflow: hidden;
}

.confidence-fill {
    height: 100%;
    background: linear-gradient(90deg, #1e5af7, #4b87ff);
    border-radius: 20px;
    transition: width 0.3s;
}

.confidence-value {
    text-align: right;
    margin-top: 5px;
    font-size: 18px;
    font-weight: bold;
    color: #0c4dba;
}

/* ---------- BACK BUTTON ---------- */
.secondary-btn {
    display: inline-block;
    margin-top: 20px;
    padding: 12px 18px;
    background: #e4eaff;
    border-radius: 12px;
    color: #0a3d7e;
    text-decoration: none;
    font-weight: 600;
}

.secondary-btn:hover {
    background: #d4ddff;
}


###  Cell : Stop Any Previously Running Flask & ngrok Processes

This cell safely **terminates any existing Flask servers and ngrok tunnels** before starting a new deployment instance.

---

####  Why This Step Is Important

If old processes are still running:
-  Flask may fail to start due to **port already in use**
-  ngrok may create **multiple conflicting tunnels**
-  You may see incorrect or cached outputs

Stopping them ensures:
 Clean server restart  
 Reliable ngrok tunnel creation  
 No port conflicts  



In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

### 🟢 Cell : Check If Port 5000 Is Already in Use

This cell checks whether **any process is currently using port 5000**, which is the default port for your Flask web application.

---

#### ✅ Why This Step Is Important

If port `5000` is already in use:
-  Flask will fail to start  
-  You may see an error like: *“Address already in use”*  
-  Your app will not be accessible  

This command helps you **identify which process is blocking the port** before launching Flask.



In [ ]:
!lsof -i :5000

### 🟢 Cell : Forcefully Terminate a Specific Process by PID

This cell forcefully **kills a specific running process** using its **Process ID (PID)**.  
It is typically used when a process is **locking the Flask port (5000)** and cannot be stopped normally.

---

#### ✅ Why This Step Is Important

If a Flask or background process:
-  Does not stop using `pkill`
-  Continues to occupy port `5000`
-  Prevents your web app from launching  

Then force-killing the process by PID ensures:
 Immediate termination  
 Port is released instantly  
 No further port conflicts  

In [ ]:
!kill -9 17471

###  Cell : Run Flask Server in the Background (Non-Blocking Mode)

This cell starts the **Flask web application in the background**, allowing the notebook to remain interactive while the server keeps running continuously.

---

####  Why This Step Is Important

Running Flask in the background allows you to:

-  Start **ngrok** in the next step  
-  Avoid blocking the notebook execution  
-  Keep the web app running without interruption  
-  Monitor logs separately  

If Flask runs in the foreground:
-  The notebook will freeze  
-  You won’t be able to launch ngrok  
-  You must manually interrupt execution  


In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &

### 🟢 Cell 9: Run Flask Server in the Background (Non-Blocking Mode)

This cell starts the **Flask web application in the background** so that:
-  The server keeps running continuously  
-  The notebook remains free for other commands  
-  You can start ngrok after this without interruption  

---

#### ✅ Why This Step Is Important

If Flask runs in the foreground:
-  The notebook will be blocked  
-  You won’t be able to run ngrok  
-  You’ll need to manually stop the server  

Running Flask in the background ensures:
 Smooth deployment workflow  
 Continuous server execution  
 Ability to monitor logs separately  

### 🔑 How to Create & Use an ngrok Authentication Token (One-Time Setup)

This step is **mandatory** to generate a public URL for your Flask app running in Google Colab.

---

####  Step 1: Create a Free ngrok Account

1. Open the official ngrok website:  
   https://ngrok.com  
2. Click **Sign Up**  
3. Sign up using:
   - Google account OR
   - Email + password  
4. Log in to your ngrok dashboard after signup.

---
   
####  Step 2: Get Your ngrok Auth Token

1. After logging in, go to:  
   **Dashboard → Your Authtoken**
2. You will see a command like this:

In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = ""  # 🔑 replace with your token

public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)


#### Used to check logs of the model while running the app . Mostly used while we face any errors

In [ ]:
# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log